In [24]:
%load_ext autoreload
%autoreload 2
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from datetime import datetime

from henpy.searchers import searchers
from henpy.persist.tables import Base
from henpy.persist import tables
from henpy.utilities.tagtools import TagManager

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import os
# Create the database connection and initialize the database
table_path = "test.sqlite3"
try:
    os.remove(table_path)
except FileNotFoundError:
    pass

engine = create_engine(f"sqlite:///{table_path}")

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

# Initialize the tag manager from an existing dump
tm = TagManager.from_file("data/tagdata.tsv")
for tag in tm.tags:
    tag_name = tag.data[tag.base_lang].name
    db_tag = tables.Tag(name=tag_name)
    db_tag.data = [tables.TagData(language=tag_datum.lang,
                                  name=tag_datum.name,
                                  display_name=tag_datum.display_name)for lang, tag_datum in tag.data.items()]
    session.add(db_tag)
session.commit()

In [35]:
# Prepopulate with a sample video and a reduced set of it's associated tags
"""
[<VideoMetadata:code=MADM-101|tags=[<Tag:data=<TagData:genre-Creampie>>, <Tag:data=<TagData:genre-Solowork>>, <Tag:data=<TagData:genre-Girl>>, <Tag:data=<TagData:genre-Beautiful Girl>>, <Tag:data=<TagData:genre-Slut>>, <Tag:data=<TagData:genre-Bride, Young Wife>>, <Tag:data=<TagData:genre-Drama>>, <Tag:data=<TagData:star-Atomi Shuri>>]>]
"""
def get_tag(tag_name, session=session):
    """Quick and dirty function to get the tag"""
    res = session.query(tables.Tag)\
                 .filter(tables.Tag.data.any(tables.TagData.name==tag_name)).first()
    if res:
        return res
    return None

tags = ["Creampie", "Solowork", "Slut"]
tags = [get_tag(i) for i in tags]
print(tags)
video = tables.Video(code="MADM-101",
                     release_date=datetime.strptime("2014-01-24", "%Y-%m-%d"),
                     image_path=None,
                     director="",
                     maker="",
                     label="",
                     tags=tags)
session.add(video)
session.commit()



[<henpy.persist.tables.Tag object at 0x0000023074B96240>, <henpy.persist.tables.Tag object at 0x0000023074925240>, <henpy.persist.tables.Tag object at 0x000002307497DE10>]


In [36]:
t1 = session.query(tables.Tag).limit(5).all()
t2 = t1[0]
t2.videos

[]

In [37]:
tm = TagManager()
s = searchers.JavlibrarySearcher(tm)




In [9]:
a = s.search("LOVE-049", return_multi=True)
b = s.search("LOVE-048", return_multi=True)
c = s.search("MADM-101", return_multi=True)


LOVE-49

2014-01-24
[<VideoMetadata:code=LOVE-48|tags=[<Tag:data=<TagData:genre-Girl>>, <Tag:data=<TagData:genre-Best, Omnibus>>, <Tag:data=<TagData:genre-Beautiful Girl>>, <Tag:data=<TagData:genre-Bukkake>>, <Tag:data=<TagData:genre-4HR+>>, <Tag:data=<TagData:star-Uehara Ai>>, <Tag:data=<TagData:star-Tsubomi>>, <Tag:data=<TagData:star-Hatsume Rina>>, <Tag:data=<TagData:star-Kasugano Yui>>, <Tag:data=<TagData:star-Aoi Koharu>>, <Tag:data=<TagData:star-Usui Aimi>>, <Tag:data=<TagData:star-Aoi Ichigo>>]>]
[<VideoMetadata:code=MADM-101|tags=[<Tag:data=<TagData:genre-Creampie>>, <Tag:data=<TagData:genre-Solowork>>, <Tag:data=<TagData:genre-Girl>>, <Tag:data=<TagData:genre-Beautiful Girl>>, <Tag:data=<TagData:genre-Slut>>, <Tag:data=<TagData:genre-Bride, Young Wife>>, <Tag:data=<TagData:genre-Drama>>, <Tag:data=<TagData:star-Atomi Shuri>>]>]


In [45]:
video_data = a[0]
print(video_data.code)
print(video_data.director)
print(video_data.release_date)
video_data.tags
video_data_2 = b[0]
print(c)
print(c[0].release_date)

LOVE-49

2014-01-24
[<VideoMetadata:code=MADM-101|tags=[<Tag:data=<TagData:genre-Creampie>>, <Tag:data=<TagData:genre-Solowork>>, <Tag:data=<TagData:genre-Girl>>, <Tag:data=<TagData:genre-Beautiful Girl>>, <Tag:data=<TagData:genre-Slut>>, <Tag:data=<TagData:genre-Bride, Young Wife>>, <Tag:data=<TagData:genre-Drama>>, <Tag:data=<TagData:star-Atomi Shuri>>]>]
2018-11-23


In [48]:
def get_videos(code, session=session):
    return session.query(tables.Video).filter(tables.Video.code==code).all()

def get_tag(tag_name, session=session):
    """Quick and dirty function to get the tag. To be replaced with nice little class method"""
    res = session.query(tables.Tag)\
                 .filter(tables.Tag.data.any(tables.TagData.name==tag_name)).first()
    if res:
        return res
    return None
    

# Check if other videos with this code exist
existing_videos = get_videos(video_data.code)
print(existing_videos)
existing_videos[0].tags
      
# # Add the video to the session and all its tags too
# v_row = tables.Video(code=video_data.code,
#                      release_date=datetime.strptime(video_data.release_date, "%Y-%m-%d"),
#                      image_path=video_data.image_path,
#                      director=video_data.director,
#                      maker=video_data.maker,
#                      label=video_data.label)




# session.add(v_row)
# session.commit()



[<VideoMetadata:code=LOVE-49>]


[]

In [58]:
v = existing_videos[0]
# Create the tags
for tag in video_data.tags:
    print(tag)
    for lang, data in tag.data.items():
        print(lang, data.name, data.type)

<Tag:data=<TagData:genre-Creampie>>
en Creampie genre
<Tag:data=<TagData:genre-Girl>>
en Girl genre
<Tag:data=<TagData:genre-Voyeur>>
en Voyeur genre
<Tag:data=<TagData:genre-Bukkake>>
en Bukkake genre
<Tag:data=<TagData:genre-Rape>>
en Rape genre
<Tag:data=<TagData:star-Aoi Ichigo>>
en Aoi Ichigo star


In [37]:
a = session.query(tables.Video).all()[0]
a.code

'LOVE-49'

In [6]:

class VideoTagManager:
    def __init__(self, db_path):
        self.session = _init_db(db_path)
        
    def _init_db(self, db_path):
        engine = create_engine("sqlite:///" + db_path)
        Session = sessionmaker(bind=engine)
        session = Session()
        return session()
    
    def get_tag(self, tag_name):
        pass
    
    def get_videos(self, video_code):
        pass
    
    def add_video(self, md):
        """Adds a video to the database
        @args
            md (VideoMetaData object from henpy searcher)
        """
        # Check for exisiting 
        tables.Video(code=md.code,
                     release_date=md.release_date,
                     image_path=md.image_path,
                     director=md.director,
                     maker=md.maker,
                     label=md.label)

        

In [21]:
session.query(tables.Tag)\
       .filter(tables.Tag.data.any(tables.TagData.name=="Creampie"))

'<henpy.persist.tables.Tag object at 0x0000023074925B38>'